<a href="https://colab.research.google.com/github/vitorfornaro/One-Billion-Row-Challenge-Python/blob/main/QB_INGESTION_PT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Este Colab fue desarrollado para conectar y extraer datos de las tablas faturacion, deposito, liquidaciones, pagos_a_banco y credit_repairs de la API de QuickBase, relacionadas con las actividades financieras en Portugal. Los datos son procesados y cargados automáticamente en una hoja de Google Sheets, organizados en pestañas correspondientes para facilitar el análisis.

*Analista responsable: Vitor Fornaro* | vitor.fornaro@gobravo.pt

Tablas extraídas:

	•	Faturacion: Información sobre facturación y cobros.
	•	Deposito: Registros de depósitos financieros.
	•	Liquidaciones: Datos sobre liquidaciones y acuerdos financieros.
	•	Pagos a Banco: Detalles de pagos realizados a instituciones bancarias.
	•	Credit Repairs: Información sobre reparaciones de crédito.



In [54]:
import openpyxl
import gspread
import requests
import json
import pandas as pd
import time

from google.colab import auth, drive
from cryptography.fernet import Fernet
from google.oauth2 import service_account
from gspread_dataframe import get_as_dataframe, set_with_dataframe
from google.auth import default
from requests.exceptions import RequestException

auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)
drive.mount('/content/gdrive')


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [55]:
token = 'b9hjmb_nuvr_0_bjckmq3bwm8vsnn438swxffuif'
realm = 'https://sebix.quickbase.com'

tables_id = {
    "faturacion": "bqxyz35cb",
    "deposito": "bqxyz35bz",
    "liquidaciones": "bqxyz35c3",
    "pagos_a_banco": "bqxyz35c8",
    "credit_repairs": "bqxyz35br"
}

headers = {
    'QB-Realm-Hostname': f'{realm}',
    'Authorization': f'QB-USER-TOKEN {token}',
    'Content-Type': 'application/json'
}

def campos_tabela(table_name):
    """
    Obtém os campos de uma tabela no Quickbase.

    Args:
        table_name (str): Nome da tabela conforme definido no dicionário `tables_id`.

    Returns:
        pd.DataFrame: DataFrame com os campos da tabela, se a requisição for bem-sucedida.
    """
    table_id = tables_id.get(table_name)

    if not table_id:
        print(f"Erro: Tabela '{table_name}' não encontrada em `tables_id`.")
        return None

    params = {
        "tableId": f"{table_id}"
    }

    try:
        r = requests.get(
            f'https://api.quickbase.com/v1/fields',
            params=params,
            headers=headers
        )

        if r.status_code == 200:
            campos = r.json()
            campos_df = pd.json_normalize(campos)
            return campos_df
        else:
            print("Erro ao obter campos:", r.status_code, r.text)
            return None
    except Exception as e:
        print(f"Erro ao fazer a requisição: {e}")
        return None

# Criando um DataFrame para cada tabela no dicionário `tables_id`
df_cols = {}

for table_name in tables_id.keys():
    df = campos_tabela(table_name)
    if df is not None:
        # Armazenando o DataFrame no dicionário com o nome correspondente
        df_cols[table_name] = df
        print(f"Colunas para '{table_name}' carregadas com sucesso.")
    else:
        print(f"Erro ao carregar colunas para '{table_name}'.")

# Criando variáveis individuais para cada tabela
df_cols_faturacion = df_cols.get("faturacion")
df_cols_deposito = df_cols.get("deposito")
df_cols_liquidaciones = df_cols.get("liquidaciones")
df_cols_pagos_a_banco = df_cols.get("pagos_a_banco")
df_cols_credit_repairs = df_cols.get("credit_repairs")

Colunas para 'faturacion' carregadas com sucesso.
Colunas para 'deposito' carregadas com sucesso.
Colunas para 'liquidaciones' carregadas com sucesso.
Colunas para 'pagos_a_banco' carregadas com sucesso.
Colunas para 'credit_repairs' carregadas com sucesso.


In [56]:
for table_name, df in df_cols.items():
    if df is not None:
        print(f"Colunas disponíveis em '{table_name}':")
        print(df[['label', 'id']].to_string(index=False))

Colunas disponíveis em 'faturacion':
                label  id
           Referencia  12
 Fecha de facturacion   8
     Tipo de comision   7
                Monto   9
               Cobros  14
            Add Cobro  15
  Tipo de Comprobante  17
             Id Deuda  18
             Cobrable  19
              Balance  20
           Por Cobrar  22
        Total Cobrado  23
      Estado de Cobro  24
      Dias Por Cobrar  25
Ultima Fecha de Cobro  27
           Id Credito  28
            Unnamed 1  29
           INCOBRABLE  30
               Mov ID  31
         Date Created   1
        Date Modified   2
     Last Modified By   5
           Record ID#   3
         Record Owner   4
Colunas disponíveis em 'deposito':
                          label  id
                        Ref RTD   7
                          Fecha   8
                        Importe   9
                       Id Berex  12
                    Descripción  17
                 Ref RTD - Name  19
                  Cuenta n

In [63]:
# Listas de colunas desejadas para cada tabela
colunas_map = {
    "faturacion": [
        "Record ID#", "Tipo de Comprobante", "Fecha de facturacion",
        "Referencia", "Tipo de comision", "Monto",
        "Total Cobrado", "Por Cobrar", "Estado de Cobro"
    ],
    "deposito": [
        "Ref RTD", "Fecha", "Importe"
    ],
    "liquidaciones": [
        "Reference", "Fecha Liquidacion", "PaB acordado", "Tipo de Convenio"
    ],
    "pagos_a_banco": [
        "Referencia", "Fecha de pago", "Importe Cuota", "Banco", "ID de Deuda", "Pagado"
    ],
    "credit_repairs": [
        "Reference", "Full Name", "Status",
        "Deuda Resuelve", "Sum Deposits", "Total Facturado"
    ]
}

# Dicionários para armazenar os mapeamentos
cols_dicts = {}
cols_ids = {}

# Iterar pelas tabelas e realizar o mapeamento
for table_name, colunas_desejadas in colunas_map.items():
    df = df_cols.get(table_name)
    if df is not None:
        # Criar o dicionário de mapeamento {id: label}
        cols_dict = df.query('label in @colunas_desejadas')[['label', 'id']].set_index('id')['label'].to_dict()
        # Criar a lista de IDs
        cols_id = df.query('label in @colunas_desejadas')['id'].tolist()

        # Armazenar os resultados nos dicionários
        cols_dicts[table_name] = cols_dict
        cols_ids[table_name] = cols_id

        print(f"Mapeamento criado para '{table_name}' com sucesso.")
    else:
        print(f"Erro: Não foi possível carregar as colunas para '{table_name}'.")

# Exibir os mapeamentos criados
print("\nDicionários de mapeamento (cols_dicts):")
for table_name, cols_dict in cols_dicts.items():
    print(f"{table_name}: {cols_dict}")

print("\nListas de IDs (cols_ids):")
for table_name, cols_id in cols_ids.items():
    print(f"{table_name}: {cols_id}")

Mapeamento criado para 'faturacion' com sucesso.
Mapeamento criado para 'deposito' com sucesso.
Mapeamento criado para 'liquidaciones' com sucesso.
Mapeamento criado para 'pagos_a_banco' com sucesso.
Mapeamento criado para 'credit_repairs' com sucesso.

Dicionários de mapeamento (cols_dicts):
faturacion: {12: 'Referencia', 8: 'Fecha de facturacion', 7: 'Tipo de comision', 9: 'Monto', 17: 'Tipo de Comprobante', 22: 'Por Cobrar', 23: 'Total Cobrado', 24: 'Estado de Cobro', 3: 'Record ID#'}
deposito: {7: 'Ref RTD', 8: 'Fecha', 9: 'Importe'}
liquidaciones: {9: 'Reference', 34: 'Tipo de Convenio', 7: 'PaB acordado', 6: 'Fecha Liquidacion'}
pagos_a_banco: {19: 'Referencia', 22: 'ID de Deuda', 7: 'Fecha de pago', 13: 'Importe Cuota', 16: 'Pagado', 20: 'Banco'}
credit_repairs: {6: 'Reference', 9: 'Full Name', 57: 'Status', 11: 'Deuda Resuelve', 15: 'Sum Deposits', 112: 'Total Facturado'}

Listas de IDs (cols_ids):
faturacion: [12, 8, 7, 9, 17, 22, 23, 24, 3]
deposito: [7, 8, 9]
liquidaciones: 

# Query

In [64]:
def consulta_qb_todas():
    """
    Consulta todas as tabelas definidas no dicionário `tables_id`, normaliza os dados retornados,
    ajusta as colunas, e retorna os DataFrames processados.

    Returns:
        dict: Um dicionário onde as chaves são os nomes das tabelas e os valores são os DataFrames processados.
    """
    resultados = {}

    for table_name, table_id in tables_id.items():
        cols_id = cols_ids.get(table_name)
        colunas_desejadas = colunas_map.get(table_name)  # Obter colunas desejadas para a tabela
        if not cols_id or not colunas_desejadas:
            print(f"Erro: IDs ou colunas desejadas não definidos para a tabela '{table_name}'. Pulando...")
            continue

        query_qb = {
            "from": f"{table_id}",
            "select": cols_id
        }

        try:
            r = requests.post(
                f'https://api.quickbase.com/v1/records/query',
                json=query_qb,
                headers=headers
            )
            if r.status_code == 200:
                print(f"Consulta realizada com sucesso para a tabela '{table_name}'!")
                dados = r.json()

                # Normalizar os dados e criar DataFrame
                if 'data' in dados:
                    df = pd.json_normalize(dados['data'])

                    # Renomear colunas para simplificar os nomes
                    for col in df.columns:
                        x = f'{col}'.split('.')[0]
                        df.rename(columns={col: x}, inplace=True)

                    # Converter as colunas para IDs numéricos e renomeá-las com base no mapeamento
                    df.columns = df.columns.astype(int)
                    df.rename(columns=cols_dicts[table_name], inplace=True)

                    # Filtrar apenas as colunas desejadas
                    df = df[colunas_desejadas]
                    resultados[table_name] = df
                else:
                    print(f"A tabela '{table_name}' retornou dados vazios ou estrutura inesperada.")
                    resultados[table_name] = pd.DataFrame()  # DataFrame vazio
            else:
                print(f"Erro ao consultar a tabela '{table_name}':", r.status_code, r.text)
        except Exception as e:
            print(f"Erro ao consultar a tabela '{table_name}': {e}")

    return resultados

# Chamar a função para consultar e processar todas as tabelas
dataframes = consulta_qb_todas()

# Exibir os dados processados de cada tabela
for table_name, df in dataframes.items():
    print(f"\nDados processados da tabela '{table_name}':")
    print(df.head() if not df.empty else "Tabela retornou nenhum dado.")

Consulta realizada com sucesso para a tabela 'faturacion'!
Consulta realizada com sucesso para a tabela 'deposito'!
Consulta realizada com sucesso para a tabela 'liquidaciones'!
Consulta realizada com sucesso para a tabela 'pagos_a_banco'!
Consulta realizada com sucesso para a tabela 'credit_repairs'!

Dados processados da tabela 'faturacion':
   Record ID# Tipo de Comprobante Fecha de facturacion  Referencia  \
0           2             Factura           2020-11-13  UPT0000028   
1           4             Factura           2020-10-30  UPT0000010   
2           5             Factura           2020-10-30  UPT0000020   
3           6             Factura           2020-10-29  UPT0000002   
4           7             Factura           2020-11-09  UPT0000021   

   Tipo de comision   Monto  Total Cobrado  Por Cobrar Estado de Cobro  
0  COMISSAO INICIAL   60.09          60.09         0.0         Cobrado  
1  COMISSAO INICIAL   92.36          92.36         0.0         Cobrado  
2  COMISSAO IN

In [59]:
# Processar cada DataFrame após a consulta
for table_name, df in dataframes.items():
    if not df.empty:  # Verificar se o DataFrame não está vazio
        try:
            # Obter os mapeamentos e colunas desejadas para a tabela
            cols_dict = cols_dicts.get(table_name)
            colunas_desejadas = colunas_map.get(table_name)

            # Verificar se as colunas ainda estão no formato de IDs numéricos
            if all(col.isdigit() for col in df.columns):
                df.columns = df.columns.astype(int)  # Converter colunas para IDs numéricos
                df.rename(columns=cols_dict, inplace=True)  # Renomear colunas com base no mapeamento

            # Filtrar apenas as colunas desejadas
            df = df[colunas_desejadas]
            dataframes[table_name] = df  # Substituir o DataFrame processado no dicionário

            print(f"Tabela '{table_name}' processada com sucesso!")
        except Exception as e:
            print(f"Erro ao processar a tabela '{table_name}': {e}")
    else:
        print(f"Tabela '{table_name}' está vazia. Nenhum processamento realizado.")

# Exemplo: Exibir o DataFrame processado

Tabela 'faturacion' processada com sucesso!
Tabela 'deposito' processada com sucesso!
Tabela 'liquidaciones' processada com sucesso!
Tabela 'pagos_a_banco' processada com sucesso!
Tabela 'credit_repairs' processada com sucesso!


# Sheets

In [65]:
# IDs das tabelas no QuickBase
tables_id = {
    "faturacion": "bqxyz35cb",
    "deposito": "bqxyz35bz",
    "liquidaciones": "bqxyz35c3",
    "pagos_a_banco": "bqxyz35c8",
    "credit_repairs": "bqxyz35br"
}

# IDs das abas no Google Sheets
sheet_ids = {
    "faturacion": "483825507",
    "deposito": "727559190",
    "liquidaciones": "826713804",
    "pagos_a_banco": "2030568323",
    "credit_repairs": "1814455247"
}

# ID do Google Sheets
sheet_workbook = "1LXn7RsfddkMAPMVS3NrvZ5eLyqgG6EzIoPafDoNzqFs"

def get_sheet(workbook_id, aba_id, tentativas=10, segundos_espera=10):
    """
    Obtém uma aba específica em uma planilha do Google Sheets.

    Args:
        workbook_id (str): ID da planilha.
        aba_id (str): ID da aba desejada.
        tentativas (int): Número de tentativas.
        segundos_espera (int): Tempo de espera entre tentativas.

    Returns:
        gspread.Worksheet: Objeto representando a aba selecionada.
    """
    for tentativa in range(tentativas):
        try:
            client = gspread.authorize(creds)
            workbook = client.open_by_key(workbook_id)  # Seleção da planilha
            sheet = workbook.get_worksheet_by_id(aba_id)  # Seleção da aba pelo ID
            return sheet
        except (RequestException, gspread.exceptions.APIError, Exception) as e:
            print(f"Erro encontrado: {e}. Tentativa {tentativa + 1} de {tentativas}.")
            if tentativa < tentativas - 1:
                print(f"Aguardando {segundos_espera} segundos antes de tentar novamente...")
                time.sleep(segundos_espera)
            else:
                print("Número máximo de tentativas atingido. Abortando operação.")
                raise

# Carregar os DataFrames no Google Sheets
for table_name, df in dataframes.items():
    if not df.empty:  # Verificar se o DataFrame não está vazio
        aba_id = sheet_ids.get(table_name)
        if aba_id:
            try:
                # Obter a aba correspondente
                sheet = get_sheet(sheet_workbook, aba_id)
                # Carregar os dados no Google Sheets
                set_with_dataframe(sheet, df, include_index=False, include_column_header=True)
                print(f"Dados da tabela '{table_name}' carregados com sucesso na aba de ID '{aba_id}'.")
            except Exception as e:
                print(f"Erro ao carregar os dados da tabela '{table_name}' na aba de ID '{aba_id}': {e}")
        else:
            print(f"ID da aba não encontrado no mapa para a tabela '{table_name}'.")
    else:
        print(f"Tabela '{table_name}' está vazia. Nenhum dado carregado no Google Sheets.")

Dados da tabela 'faturacion' carregados com sucesso na aba de ID '483825507'.
Dados da tabela 'deposito' carregados com sucesso na aba de ID '727559190'.
Dados da tabela 'liquidaciones' carregados com sucesso na aba de ID '826713804'.
Dados da tabela 'pagos_a_banco' carregados com sucesso na aba de ID '2030568323'.
Dados da tabela 'credit_repairs' carregados com sucesso na aba de ID '1814455247'.
